In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Some plotting niceties
plt.rc('figure', figsize=(10, 4))
sns.set_style('white')
sns.set_style('ticks')

import nengo
import phd

def img(array):
    plt.pcolormesh(array.T)
    plt.ylim(top=array.shape[1])
    plt.xlim(right=array.shape[0])
    plt.yticks(())
    sns.despine(left=True)
    plt.tight_layout()

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

In [ ]:
model = phd.sermo.Production()
# model.sequence.syllable_d = 96
model.trial.sequence = 'PAT*POS1 + DAS*POS2 + KAP*POS3'
freqs = [('pat', 2.5), ('das', 4.7), ('kap', 3.2)]
for ges, freq in freqs:
    path = phd.ges_path('ges-de-cvc', '%s.ges' % ges.lower())
    traj = phd.vtl.parse_ges(path).trajectory(dt=model.trial.dt)
    model.add_syllable(label=ges.upper(), freq=freq, trajectory=traj)

In [ ]:
net = model.build()
with net:
    p_syll = nengo.Probe(net.sequence.syllable.output, synapse=0.01)
    p_gate = nengo.Probe(net.sequencer.gate, synapse=0.01)
    p_reset = nengo.Probe(net.sequencer.reset, synapse=0.01)
    p_timer = nengo.Probe(net.sequencer.timer, synapse=0.01)
    p_disinhib = [nengo.Probe(dmp.disinhibit, synapse=0.01) for dmp in net.syllables]
    p_dmps = [nengo.Probe(dmp.osc, synapse=0.01) for dmp in net.syllables]
    p_prod = nengo.Probe(net.production_info.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(0.924 + 0.2)

In [ ]:
syllables = [s[0].upper() for s in freqs]
plt.plot(sim.trange(), 
         nengo.spa.similarity(sim.data[p_syll],
                              net.sequence.vocab.create_subset(syllables)))
plt.legend(syllables, loc='best')

In [ ]:
plt.plot(sim.trange(), sim.data[p_gate])
plt.plot(sim.trange(), sim.data[p_reset])
plt.plot(sim.trange(), sim.data[p_timer])
plt.legend(['gate', 'kick', 'timer', 'timer'], loc="best")
# plt.ylim([-0.1, 1.1])

In [ ]:
for p_dis in p_disinhib:
    plt.plot(sim.trange(), sim.data[p_dis])
plt.legend(['PAT', 'DAS', 'KAP'], loc="best")
# plt.ylim([-0.1, 1.1])

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(sim.data[p_timer].T[0], sim.data[p_timer].T[1])
for i, p_dmp in enumerate(p_dmps):
    plt.subplot(2, 2, i + 2)
    plt.plot(sim.data[p_dmp].T[0], sim.data[p_dmp].T[1])

In [ ]:
plt.plot(sim.trange(), sim.data[p_prod]);

## Construct a gesture score from the prod data

In [ ]:
# Target: actual trajectories
from phd.experiments import shorten

full_traj = []
order = [0, 1, 2]
for i in order:
    syll = model.syllables[i]
    speed = syll.freq
    t_frames = int((1. / speed) / model.trial.dt)
    full_traj.append(shorten(syll.trajectory, t_frames))
full_traj = np.vstack(full_traj)
img(full_traj)
print full_traj.shape[0]

In [ ]:
delay_frames = 200
y = sim.data[p_prod][delay_frames:]
print(y.shape)
img(y)

In [ ]:
import nengo.utils.numpy as npext
print(npext.rmse(full_traj, y))

In [ ]:
# Make a gesture score out of the traj
from phd.mfcc import derivative

tderiv = np.abs(derivative(y, 20))
slices = tderiv > 0.015
img(full_traj)
plt.figure()
img(slices)

In [ ]:
x_ind, y_ind = np.where(np.abs(np.diff(np.vstack([np.zeros(48), slices]), axis=0)))
# Add a blank area at the start, so that the starting neutral gesture is recorded
x_ind = np.hstack([np.zeros(48 * 2, dtype=int), x_ind])
y_ind = np.hstack([np.arange(48), np.arange(48), y_ind])
img(slices)
plt.plot(x_ind, y_ind, ls='none', marker='o')

In [ ]:
# Sort by y_index then x_index as each gesture depends on the two
# subsequent time slices for that gesture
sort_ix = np.argsort(y_ind)
x_ind = x_ind[sort_ix]
y_ind = y_ind[sort_ix]

# sort within each group
for yi in np.unique(y_ind):
    x_ind[y_ind==yi] = np.sort(x_ind[y_ind==yi])

print y_ind
print x_ind

In [ ]:
def gest2seqlabel(gesture, vtl=None):
    if vtl is None:
        vtl = phd.vtl.VTL()
    labels = vtl.gesture_labels()
    labels.remove("f0")
    ix = labels.index(gesture)
    return ix2seqlabel(ix, labels)

def ix2seqlabel(ix, labels):
    if ix < labels.index('ll-labial-nas'):
        return 'vowel-gestures'
    elif ix < labels.index('tt-alveolar-nas'):
        return 'lip-gestures'
    elif ix < labels.index('tb-palatal-fric'):
        return 'tongue-tip-gestures'
    elif ix < labels.index('breathy'):
        return 'tongue-body-gestures'
    elif ix < labels.index('velic'):
        return 'glottal-shape-gestures'
    elif ix < labels.index('lung-pressure'):
        return 'velic-gestures'
    else:
        return 'lung-pressure-gestures'

print phd.vtl.VTL().gesture_labels()

In [ ]:
# Make a gesture score
vtl = phd.vtl.VTL()
gs = phd.vtl.GestureScore(vtl.gesture_labels())
gs.labels.remove('f0')
seq = phd.vtl.GestureSequence('vowel-gestures')
gs.sequences.append(seq)
for i in range(0, x_ind.size, 2):
    this_start = x_ind[i]
    this_end = x_ind[i+1]

    if i+3 > x_ind.size:
        next_start = next_end = tderiv.shape[0]
    else:
        this_y = y_ind[i]
        next_y = y_ind[i+2]
        if this_y != next_y:
            next_start = next_end = tderiv.shape[0]
        else:
            next_start = x_ind[i+2]
            next_end = x_ind[i+3]

    seqlabel = ix2seqlabel(this_y, gs.labels)
    if seq.type != seqlabel:
        seq = phd.vtl.GestureSequence(seqlabel)
        gs.sequences.append(seq)

    value = np.mean(y[this_end:next_start, this_y])
    if value < 0.05:
        neutral = True
    else:
        neutral = False
    value = gs.labels[this_y]
    
    tau = (this_end - this_start) * model.trial.dt * 0.5  # to match normal values
    duration = (next_start - this_start) * model.trial.dt
    if duration < 1e-0.:
        continue
    seq.gestures.append(phd.vtl.Gesture(value, 0., duration, tau, neutral))

img(gs.trajectory(dt=model.trial.dt))
for seq in gs.sequences:
    print "---", seq.type
    for ges in seq.gestures:
        print ges.value, ges.duration_s, ges.neutral